In [2]:
import numpy as np
from scipy import spatial
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import os,sys
my_path = os.getcwd()
print (my_path)
path = os.path.join(my_path, "../data/ratings.txt")
print (path)

/home/surya/surya_114cs0121/AttackProfileDetectionRS/filmtrust/Item-Based-Recommendation
/home/surya/surya_114cs0121/AttackProfileDetectionRS/filmtrust/Item-Based-Recommendation/../data/ratings.txt


In [4]:
ratings = pd.read_csv(path,sep=' ',header=None,names=['userId','itemId','rating'])

In [5]:
ratings.describe()

,userId,itemId,rating
count,35497.000000,35497.000000,35497.000000
mean,744.247824,265.524946,3.002803
std,433.199457,346.854769,0.918692
min,1.000000,1.000000,0.500000
25%,356.000000,12.000000,2.500000
50%,735.000000,216.000000,3.000000
75%,1130.000000,251.000000,4.000000
max,1508.000000,2071.000000,4.000000


### Rating before profile injection

In [6]:
calculated_Rating = {}
total_Ratings = {}
all_items = ratings['itemId'].unique()

#rating
R = ratings.groupby(['itemId'])['rating'].mean()
for i in all_items:
#     print (i)
    calculated_Rating[i] = R[i]
    total_Ratings[i] = len(ratings[ratings['itemId']==i])
    
# print (calculated_Rating)

In [7]:
calculated_Rating[7]

3.156609195402299

### Inject Attack Profiles

In [8]:
window_items =  [256, 1, 257, 6, 10, 12, 205, 206, 207, 210, 84, 212, 213, 214, 216, 217, 220, 121, 233, 234, 235, 239, 241, 243, 244, 245, 246, 247, 248, 249, 251, 252, 253, 254, 255] 
target_items = [2, 3, 4, 5, 7, 8, 9, 11, 13, 17, 211, 215, 219, 236, 250]

In [9]:
nuke_rating = 0.5

In [10]:
attack_ids = list(range(1600,1700))

for attacker in attack_ids:
    for item in window_items:
        a = calculated_Rating[item]
        attack_rating = int(a) if (a-int(a)<0.5) else int(a)+1
        ratings = ratings.append(pd.Series([attacker,item,attack_rating], index=['userId', 'itemId' ,'rating']), ignore_index=True)

for attacker in attack_ids:
    for item in target_items:
        ratings = ratings.append(pd.Series([attacker,item,nuke_rating], index=['userId', 'itemId' ,'rating']), ignore_index=True)

### Rating after profile injection

In [11]:
calculated_Rating = {}
total_Ratings = {}
all_items = ratings['itemId'].unique()

#rating
R = ratings.groupby(['itemId'])['rating'].mean()
for i in all_items:
#     print (i)
    calculated_Rating[i] = R[i]
    total_Ratings[i] = len(ratings[ratings['itemId']==i])
    
# print (calculated_Rating)

In [12]:
calculated_Rating[7]

2.924388111888112

### Reading trust values

In [13]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import h5py
warnings.resetwarnings()
with h5py.File('trust_values.h5', 'r') as hf:
    trust_val = hf['trust'][:]

## Trust propagated deviation

In [14]:
users = attack_ids[:1] 
print (users)

[1600]


In [15]:
trust_threshold = 0.9
deviation_threshold = 1

t1 = time.time()
for u in users:
    temp = ratings[ratings['userId']==u]
    items = temp['itemId'].unique()
    items.sort()
#     print (items)
    user_trust = trust_val[u][0:]
    for item in items[:1]:
        user_who_rated_same_item = [int(i) for i in list(ratings[ratings['itemId']==item]['userId'])]
        consideredUsers = np.where(user_trust>trust_threshold)[0]
#         print (user_who_rated_same_item)
#         print (consideredUsers)
        trustee = [i for i in consideredUsers if i in user_who_rated_same_item]
        trustee_trust = [user_trust[i] for i in trustee]
        trustee_rating = [ratings[(ratings['itemId']==item) & (ratings['userId']==i)].reset_index(drop=True)['rating'][0] for i in trustee]
        done_trustee = []
        for t_i, t_u in enumerate(trustee):
            if t_u in done_trustee:
                continue
            else:
                done_trustee.append(t_u)
                
                temp_user_trust = trust_val[t_u][0:]
                temp_trust = trustee_trust[t_i]
                temp_consideredUsers = np.where(temp_trust*trust_val>trust_threshold)[0]
                temp_trustee = [i for i in temp_consideredUsers if i in user_who_rated_same_item]
#                 temp_trustee = [t_l[1][i] for i in range(len(t_l[0])) if t_l[0][i]==u]
                trustee.extend(temp_trustee)
    
                temp_trustee_trust = [temp_user_trust[i] for i in temp_trustee]
                trustee_trust.extend(temp_trustee_trust)
                
                temp_trustee_rating = [ratings[(ratings['itemId']==item) & (ratings['userId']==i)].reset_index(drop=True)['rating'][0] for i in temp_trustee]
                trustee_rating.extend(temp_trustee_rating)
                
            if(t_i==len(trustee)):
                break
        
        if sum(trustee_trust)!=0:
            rating_user_network = sum([trustee_trust[p]*q for p,q in enumerate(trustee_rating)])/sum(trustee_trust)
        else:
            rating_user_network = 0
            
        if (total_Ratings[item]-len(trustee_rating)) !=0:
            temp_calculated_Rating = (calculated_Rating[item]*total_Ratings[item]-sum(trustee_rating))/(total_Ratings[item]-len(trustee_rating))
        else:
            temp_calculated_Rating =0
        
        if(abs(temp_calculated_Rating-rating_user_network)>deviation_threshold):
            print ('Attack profile detected :', u ,' for item :',item,' with Deviation : ',abs(temp_calculated_Rating-rating_user_network))
        print (abs(temp_calculated_Rating-rating_user_network))

print (time.time()-t1)

KeyboardInterrupt: 

In [33]:
import random
t = []
for i in range(5):
    temp = []
    for j in range(5):
        temp.append(random.randint(1,10)/10)
    t.append(temp)
print (t)
t = np.array(t)
l = np.where(1.5*t>0.5)
print ([l[1][i] for i in range(len(l[0])) if l[0][i]==1 ])

[[0.5, 1.0, 0.9, 0.8, 0.4], [1.0, 0.3, 1.0, 0.1, 0.4], [0.4, 0.6, 1.0, 0.5, 0.6], [1.0, 1.0, 0.5, 0.3, 0.5], [0.9, 0.6, 0.4, 0.4, 0.8]]
[0, 2, 4]


In [43]:
a =[3,3]
for j,i in enumerate(a):
    print (j)
    if len(a)==j:
        break
        
    if i<8:
        a.append(i*2)
        print (j,'  ','add')
    else:
        pass
    
print (a)

0
0    add
1
1    add
2
2    add
3
3    add
4
5
[3, 3, 6, 6, 12, 12]


In [35]:
a

[3, 2, 1, 4, 5]